### Import libraries

In [216]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import matplotlib.image as mpimg
import glob
import os

## Prepare the data

In [217]:
# Model / data parameters
img_width, img_height = 28, 28
epochs = 15
batch_size = 16
input_shape = (28, 28, 1)
num_classes = 10

notMNIST_load_data is a function that loads the notMNIST dataset. It returns three tuples containing the training, validation and test data. Each tuple is formed by a numpy array containing the images and a numpy array containing the labels (0 to 9).

In [218]:
def notMNIST_load_data() : 
    data_dir_letters = 'data/notMNIST_small'

    nb_letters = len(os.listdir(data_dir_letters))
    if nb_letters != num_classes:
        raise ValueError('The number of classes is not equal to the number of letters in the folder')

    x_train = []
    y_train = []
    x_test = []
    y_test = []

    # we get the number of samples in the smallest class
    min_nb_samples = float('inf')
    for letter in os.listdir(data_dir_letters):
        nb_samples = len(os.listdir(os.path.join(data_dir_letters, letter)))
        min_nb_samples = min(min_nb_samples, nb_samples)

    # 80% of the data is used for training
    # 10% for validation
    # 10% for testing
    nb_train_samples = int(min_nb_samples * 0.8)
    # nb_validation_samples = int(min_nb_samples * 0.1)
    nb_validation_samples = 0
    nb_test_samples = nb_samples - nb_train_samples - nb_validation_samples

    # for each letter folder, we copy the images in the train, validation or test tuple and the label
    # TODO : randomize the order of the images would be better ?
    for letter in os.listdir(data_dir_letters):
        index = 0

        for image in glob.iglob(os.path.join(data_dir_letters, letter, "*.png")):

            if index < nb_train_samples:
                pixels_array = mpimg.imread(image)
                x_train.append(pixels_array)
                y_train.append(ord(letter) - 65)
            #elif index < nb_train_samples + nb_validation_samples:
                
            elif index < nb_train_samples + nb_validation_samples + nb_test_samples:
                pixels_array = mpimg.imread(image)
                x_test.append(pixels_array)
                y_test.append(ord(letter) - 65)
            index += 1
    
    return (np.array(x_train), np.array(y_train)), (np.array(x_test), np.array(y_test))

In [219]:
# load the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = notMNIST_load_data()

In [220]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [221]:
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (14970, 28, 28, 1)
14970 train samples
3750 test samples


In [222]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Build the model

In [223]:
def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [224]:
def build_model_LeNet():
    # LeNet-5 architecture
    model = Sequential()
    
    model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(AveragePooling2D())

    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
    model.add(AveragePooling2D())

    model.add(Flatten())

    model.add(Dense(units=120, activation='relu'))
    model.add(Dense(units=84, activation='relu'))
    model.add(Dense(units=num_classes, activation = 'softmax'))

    return model

In [225]:
def build_model_LeNet_new():
    # LeNet-5 architecture
    model = Sequential()
    
    model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='tanh', input_shape=input_shape))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='tanh'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())

    model.add(Dense(units=120, activation='tanh'))
    model.add(Dense(units=84, activation='tanh'))
    model.add(Dense(units=num_classes, activation = 'softmax'))

    return model

In [1]:
# get the model
model = build_model_LeNet_new()
model.summary()

NameError: name 'build_model_LeNet_new' is not defined

In [227]:
# Compile the model
model.compile(
    loss="categorical_crossentropy", 
    optimizer="rmsprop", 
    metrics=["accuracy"]
)

## Train the model

In [228]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15


843/843 [==============================] - 5s 4ms/step - loss: 1.0596 - accuracy: 0.6654 - val_loss: 9.1346 - val_accuracy: 0.0000e+00
Epoch 2/15
843/843 [==============================] - 3s 3ms/step - loss: 0.5684 - accuracy: 0.8510 - val_loss: 11.7978 - val_accuracy: 0.0000e+00
Epoch 3/15
843/843 [==============================] - 3s 3ms/step - loss: 0.4694 - accuracy: 0.8677 - val_loss: 11.4186 - val_accuracy: 0.0000e+00
Epoch 4/15
843/843 [==============================] - 2s 3ms/step - loss: 0.4153 - accuracy: 0.8780 - val_loss: 10.4892 - val_accuracy: 0.0000e+00
Epoch 5/15
843/843 [==============================] - 3s 3ms/step - loss: 0.3814 - accuracy: 0.8840 - val_loss: 10.1729 - val_accuracy: 0.0000e+00
Epoch 6/15
843/843 [==============================] - 3s 3ms/step - loss: 0.3526 - accuracy: 0.8959 - val_loss: 10.3835 - val_accuracy: 0.0000e+00
Epoch 7/15
843/843 [==============================] - 3s 3ms/step - loss: 0.3280 - accuracy: 0.9003 - val_loss: 9.7597 - val_accur

## Evaluate the trained model

In [229]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.2189500331878662
Test accuracy: 0.81413334608078


### Results - used to find the best architecture and hyperparameters
* Architecture LeNet-5 \
50 epochs, batch size 128 \
Epoch 50/50 - loss: 0.3308 - accuracy: 0.8945 \
Test loss: 2.313185691833496 \
Test accuracy: 0.649066686630249 \

* Architecture LeNet-5 \
15 epochs, batch size 16 \
Epoch 15/15 - loss: 0.4077 - accuracy: 0.8687 \
Test loss: 2.001655101776123 \
Test accuracy: 0.7760000228881836 \

* Architecture LeNet-5 \
15 epochs, batch size 16 \
Convolution : kernel_size=(5, 5), activation='relu' \
Epoch 15/15 - loss: 0.2376 - accuracy: 0.9270 \
Test loss: 1.7583626508712769 \
Test accuracy: 0.8191999793052673 \

* Architecture LeNet-5_new \
15 epochs, batch size 16 \
Convolution : kernel_size=(5, 5), activation='tanh' \
Pooling : pool_size=(2, 2), strides=(2, 2) \
Epoch 15/15 - loss: 0.2376 - accuracy: 0.9270 \
Test loss: 1.7583626508712769   \
Test accuracy: 0.8191999793052673 \